In [ ]:
!pip install -U --user numpy

In [ ]:
!pip install wget

In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import urllib
import wget

MAX_LEN = 512
EPOCHS = 3
VERBOSE = 2
BATCH_SIZE = 16

In [ ]:
DATA_OUT_PATH = './data_out/KOR'

In [ ]:
def plot_graphs(history, string, string_1, string_2):
    # loss 
    plt.plot(history.history[string])
    plt.plot(history.history[string_1])
    plt.plot(history.history[string_2])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, string_1, string_2])
    plt.show()

In [ ]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", lowercase=False)
save_path = "bert-base-multilingual-cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-cased/vocab.txt", lowercase=False)

In [ ]:
train_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_train.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', out='./bert-base-multilingual-cased/')

In [ ]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-config.json', './bert-base-multilingual-cased/config.json')

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5', out='./bert-base-multilingual-cased/')

In [ ]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5', './bert-base-multilingual-cased/tf_model.h5')

In [ ]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

#         print('context=',context)
#         print('question=',question)
#         print('answer_text=',answer_text)
#         print('start_char_idx=',start_char_idx)
        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
#         print('end_char_idx=',end_char_idx)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

#         print('tokenized_context.offsets=',tokenized_context.offsets)
        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
#         print('tokenized_context.ids=', tokenized_context.ids)
#         print('tokenized_question.ids=', tokenized_question.ids)
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
#         print('input_ids=', input_ids)
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
#         print('token_type_ids=', token_type_ids)
        attention_mask = [1] * len(input_ids)
#         print('attention_mask=', attention_mask)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


In [ ]:
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

In [ ]:
idx = 10
# print(x_train[0][0])
context = tokenizer.decode(x_train[0][idx])
print(context)

tokenized_context = tokenizer.encode(context)
offset = tokenized_context.offsets

start = y_train[0][idx]
end = y_train[1][idx]
print(start, end)

start = offset[start][0]
end = offset[end][1]

print(start, end)
print(context[start:end])


In [ ]:
class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False) #(N,H)(H,1)=(N,1)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
#         print(embedding.shape)
        start_logits = self.start_logit(embedding)
#         print(start_logits.shape)
        start_logits = self.flatten(start_logits)
#         print(start_logits.shape)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

In [ ]:
# korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
# # inputs = (x_train[0][0] , x_train[1][0], x_train[2][0])
# korquad_model(x_train)

In [ ]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [ ]:
def normalized_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [ ]:
class ExactMatch(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalized_answer(pred_ans)
            normalized_true_ans = normalized_answer(squad_eg.answer_text)
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)

In [ ]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [ ]:
model_name = "tf2_bert_korquad"

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
history = korquad_model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,  # For demonstration, 3 epochs are recommended
    verbose=VERBOSE,
    batch_size=BATCH_SIZE,
    callbacks=[exact_match_callback, cp_callback]
)

In [ ]:
print(history.history)

In [ ]:
plot_graphs(history, 'loss', 'output_1_loss', 'output_2_loss')

In [ ]:
korquad_model.save_weights('data_out/KOR/tf2_bert_korquad/weights_KorQuAD.h5')

In [ ]:
korquad_model.load_weights('data_out/KOR/tf2_bert_korquad/weights_KorQuAD.h5')

In [ ]:
korquad_model.loss

In [ ]:
plot_graphs(korquad_model.history,'loss', 'output_1_loss', 'output_2_loss')

### 모델 predict 구현

In [45]:
korquad_model.load_weights('data_out/KOR/tf2_bert_korquad/weights_KorQuAD.h5')

#### 지문 찾아보기

In [47]:
idx = 10
print(tokenizer.decode(x_eval[0][idx]))

" 내각과 장관들이 소외되고 대통령비서실의 권한이 너무 크다 ", " 행보가 비서 본연의 역할을 벗어난다 " 는 의견이 제기되었다. 대표적인 예가 10차 개헌안 발표이다. 원로 헌법학자인 허영 경희대 석좌교수는 정부의 헌법개정안 준비 과정에 대해 " 청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 했다 " 고 지적했다.'국무회의의 심의를 거쳐야 한다'( 제89조 ) 는 헌법 규정에 충실하지 않았다는 것이다. 그러면서 " 법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다 " 고 지적했다. 민정수석은 국회의원에 대해 책임지는 법무부 장관도 아니고, 국민에 대해 책임지는 사람도 아니기 때문에 정당성이 없고, 단지 대통령의 신임이 있을 뿐이라는 것이다. 또한 국무총리 선출 방식에 대한 기자의 질문에 " 문 대통령도 취임 전에 국무총리에게 실질적 권한을 주겠다고 했지만 그러지 못하고 있다. 대통령비서실장만도 못한 권한을 행사하고 있다. " 고 답변했다. 법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?


In [48]:
print(x_eval[0][idx])
print(x_eval[1][idx])
print(x_eval[2][idx])

[   101    107   8996  66540  11882   9657  20595  20173   9448  78705
  29208  70672  29455  12424  31503  10459   8917  11102  10739   9004
  32537   9834  11903    107    117    107   9966  30005  11287   9379
  12424   9358  25486  10459  69144   9342  54305  11903    107   9043
   9637 118634  10739   9672  12310  13628    119   9069  37824  15387
   9576  11287  10150  23466   8857 119426  34951   9323  37824  11925
    119   9612  11261   9969  33768 109522  12030   9968  30858   8885
  49515  14423   9426 119215  25242  93835   9670  43875   9969  33768
  21789  16605  34951   9691  29455   8898  77763  33378    107   9751
  12638  14423   9379  12424  31503  10739  63783   8909  32537  56356
  75109   9638 118895 119210  12965  21711  23622    107   8888   9706
  14801  12490    119    112   8909  32537  56356  10459   9491  78505
  69642  21711  16139    112    113   9672  11396  11373  20626    114
   9043   9969  33768   8922  77763   9770  31503  23665  49137  11018
  2419

In [49]:
tokenized_context= tokenizer.encode("법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?")

In [51]:
context_token_to_char = tokenized_context.offsets
print(len(context_token_to_char))
print(context_token_to_char)

43
[(0, 0), (0, 1), (1, 2), (2, 3), (4, 5), (5, 6), (6, 7), (8, 9), (9, 10), (10, 11), (11, 12), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (19, 20), (20, 21), (21, 22), (22, 23), (24, 25), (25, 26), (26, 28), (29, 30), (31, 32), (32, 33), (33, 34), (35, 36), (37, 39), (39, 40), (41, 42), (42, 43), (43, 44), (45, 46), (46, 47), (47, 48), (49, 50), (50, 51), (51, 52), (52, 53), (54, 57), (57, 58), (0, 0)]


In [52]:
for i in range(len(context_token_to_char)):
    print(tokenizer.decode([tokenized_context.ids[i]]), end=" ")

 법 ##무 ##부 장 ##관 ##을 제 ##쳐 ##놓 ##고 민 ##정 ##수 ##석 ##이 개 ##정 ##안 ##을 설 ##명 ##하는 게 이 ##해 ##가 안 된다 ##고 지 ##적 ##한 경 ##희 ##대 석 ##좌 ##교 ##수 이름은 ?  

In [74]:
class SquadExample_pred:
    def __init__(self, question, context):#, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        #answer_text = self.answer_text
        #start_char_idx = self.start_char_idx

#         print('context=',context)
#         print('question=',question)
        
        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        
#         print('context=',context)
#         print('question=',question)

        # Tokenize context
        tokenized_context = tokenizer.encode(context)
#         print("tokenized_context.ids=", tokenized_context.ids)
#         print("tokenized_context.ids.len=", len(tokenized_context.ids))

        # Tokenize question
        tokenized_question = tokenizer.encode(question)
#         print("tokenized_question.ids=", tokenized_question.ids)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
#         print("input_ids=", input_ids)
        
        
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
#         print("token_type_ids=", token_type_ids)
        
        attention_mask = [1] * len(input_ids)
#         print("attention_mask=", attention_mask)
        

        # Pad and create attention masks.
        # Skip if truncation is needed
        
        padding_length = MAX_LEN - len(input_ids)
        print("padding_length=",padding_length)
        
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        
#         print("input_ids=", input_ids)
#         print("token_type_ids=", token_type_ids)
#         print("attention_mask=", attention_mask)

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets
#         print(self.context_token_to_char)
        
    def get_input_target(self):
        dataset_dict = {
            "input_ids": [],
            "token_type_ids": [],
            "attention_mask": [],
        }
        if self.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(self, key))
        for key in dataset_dict:
            dataset_dict[key] = np.array(dataset_dict[key])

        x = [
            dataset_dict["input_ids"],
            dataset_dict["token_type_ids"],
            dataset_dict["attention_mask"],
        ]
#         print(x[0].shape)
        return x

def create_squad_examples_from_arg(question, context):#, start_char_idx, answer_text):
    squad_eg = SquadExample_pred(
        question, context#, start_char_idx, answer_text
    )
    squad_eg.preprocess()
    return squad_eg


def predict_test(model, pred_raw):
    x_pred = pred_raw.get_input_target()
    pred_start, pred_end = model.predict(x_pred)
#     print(pred_start.shape)
#     print(pred_end.shape)
#     print(pred_start)
    pred_start_offset_index = np.argmax(pred_start)
    pred_end_offset_index = np.argmax(pred_end)
    print(pred_start_offset_index, pred_end_offset_index)
#     print(pred_raw.context_token_to_char)
    pred_start_offset = pred_raw.context_token_to_char[pred_start_offset_index]
    pred_end_offset = pred_raw.context_token_to_char[pred_end_offset_index]
    print(pred_start_offset, pred_end_offset)

    answer = pred_context[pred_start_offset[0]:pred_end_offset[1]]
    
    normalized_pred_ans = normalized_answer(answer)
    
    return normalized_pred_ans


pred_context = '''정부가 이산화탄소 스트림의 수출을 가능하게 하는 런던의정서개정(2009년 개정) 수락서를 국제해사기구(IMO) 사무국에 기탁할 예정이라고 29일 밝혔다. 이산화탄소 스트림은 제철소나 발전소 등에서 포집한 이산화탄소다.
1996년 채택된 런던의정서는 자국 해역에서 이산화탄소 스트림 격리(저장)는 허용하는 반면, 이산화탄소 스트림 수출(국가 간 이동)은 금지했다. 
2009년 당사국총회에서 일정 절차에 따라 이산화탄소 스트림 수출을 허용하는 개정안이 채택됐다. '''
pred_question = "이산화탄소 스트림 수출(국가 간 이동)은 금지하는 협정서는?"

pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 310
88 92
(132, 133) (136, 138)
런던의정서는


In [77]:
pred_context = "1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다."
# pred_question = "1989년 6월 30일 평양축전에 대표로 파견 된 인물은?"
# pred_question = "임종석의 사전구속영장을 발부받은 날짜는?"
pred_question = "임수경이 평양축전에 파견된 연도는?"
pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 206
41 41
(65, 70) (65, 70)
1989년
